## Variables are Not Boxes
Every **object** has an **identity**, a **type** and a **value**. An object’s identity never changes once it has been created; you may think of it as the object’s address in memory.
- The `is` operator compares the **identity** of two objects
- The `id()` function returns an integer representing its **identity**.
- The `==` operator compares the **values** of objects, `a == b` invokes `a.__eq__(b)`

Variables and Objects are different in Python! It’s better to think of Python Variables as **labels attached to objects**.

### Aliases

- Aliasing: **An object** has several labels assigned to it


In [1]:
a = [1, 2, 3]
b = a
b is a, id(a) == id(b) # `is` means these two variables are attachted to the same object

(True, True)

In [2]:
a.append(4) # `append` method does not change the identity
b

[1, 2, 3, 4]

### Note: assignment operator `=` will change the **identity**, but `+=` does not necessarily change the **identity**
[Augumented Assignment](./02_An_Array_of_Sequences.ipynb#Augumented_Assignment_with_Sequences) does not change the `id` of mutable sequence, but changes the `id` of immutable sequence, because it creates a new object

In [3]:
a = [1, 2, 3]
b = a
print(a is b)
b += [4]
print(a is b)
b = [1, 2, 3, 4]
print(a is b)

True
True
False


`c` and `d` are bound to **distinct objects with the same value**, `d` is not an alias for `c` 

In [4]:
c = [1, 2, 3]
d = [1, 2, 3]
c == d, c is not d, id(c) == id(d)

(True, True, False)

### The relative Immutability of Tuples
Tuples, like most Python collections—lists, dicts, sets, etc.— hold **references to objects**.

Immutability of Tuples means **the identity of elements can not be changed**. But in some case, **the value of an object can vary without changing its identity**

In [5]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print(t1 is not t2)
old_id = id(t1[-1])
t1[-1].append(50)
new_id = id(t1[-1])
old_id == new_id, t1 == t2

True


(True, False)

## Copies are shallow by default
Using the constructor or `[:]` produces a shallow copy (i.e., the outermost container is duplicated, but the copy is filled with **references to the same items held by the original container**). 

D.h. The identity of the sequence is different from its copy, but the identities of their items are the same

In [6]:
l1 = [1, [1, 2]]
l2 = list(l1)
l3 = l1[:]
l1 is l2, l1[1] is l2[1], l1 == l2 == l3

(False, True, True)

In [7]:
# Appending to the container has no effect to its copy
# But appending one of the mutable items has effect to its copy
l1.append(100)
l1[1].append(3) # l1[1] is an alias for l2[1]
l1, l2, l3

([1, [1, 2, 3], 100], [1, [1, 2, 3]], [1, [1, 2, 3]])

### Deep and Shallow Copies of Arbitrary Objects using `deepcopy()` and `copy()`
Deep copies: duplicates that **do not share references of embedded objects**

In [8]:
import copy
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print(id(bus1), id(bus2), id(bus3))
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))
bus1.drop('Bill') # bus2 also drops Bill
bus2.passengers, bus3.passengers

139688849697280 139688849697376 139688849697616
139688849644736 139688849644736 139688849642048


(['Alice', 'Claire', 'David'], ['Alice', 'Bill', 'Claire', 'David'])

You can control the behavior of both `copy` and `deepcopy` by implementing the `__copy__()` and `__deepcopy__()` special methods

## Function Parameters as References
The only mode of parameter passing in Python is **call by sharing**. The parameters inside the function become **aliases of the actual arguments**. The result of this scheme is that a function may change any **mutable object**(`list` etc.) passed as a parameter, but it **cannot change the identity of those objects**. If parameters are forced to change their `id`s, a new object will be created. 

In [9]:
def f(a, b):
    a += b
    return a
x, y = 1, 2
f(x, y), x, y

(3, 1, 2)

## Note: assignment operator `=` will change the **identify**, but `+=` does not necessarily change the **identify**
[Augumented Assignment](./02_An_Array_of_Sequences.ipynb#Augumented_Assignment_with_Sequences) does not change the `id` of mutable sequence, but changes the `id` of immutable sequence, because it creates a new object

In [10]:
l1 = [1, 2, 3]
l2 = [4]
f(l1, l2), l1, l2 # Augumented Assignment does not change the `id` of mutable sequence

([1, 2, 3, 4], [1, 2, 3, 4], [4])

In [11]:
t1 = (1, 2)
t2 = (3, 4)
f(t1, t2), t1, t2 # Augumented Assignment changes the `id` of immutable sequence, because it creates a new object

((1, 2, 3, 4), (1, 2), (3, 4))

### Mutable Types as Parameter Defaults: Bad Idea
Let's see an example of haunted bus:

In [12]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    # Use Mutable types as Parameter defaults
    def __init__(self, passengers=[]):
        # This assignment makes `self.passengers` an alias for passengers, which is itself an alias for the default list, when no `passengers` argument is given. 
        self.passengers = passengers  

    def pick(self, name):
        self.passengers.append(name)  

    def drop(self, name):
        self.passengers.remove(name)

bus1 = HauntedBus()
bus1.pick('Carrie')
bus2 = HauntedBus()
bus2.passengers, bus1.passengers is bus2.passengers # same identity

(['Carrie'], True)

### Defensive Programming with Mutable Parameters


In [13]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  
        else:
            self.passengers = passengers # The bus is aliasing the list that is passed to the constructor.

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team # `list` object will also be changed

['Sue', 'Maya', 'Diana']

In [14]:
class TwilightBus:
    def __init__(self, passengers=None):
            if passengers is None:
                self.passengers = []
            else:
                self.passengers = list(passengers) # shallow copy using the constructer
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)
        
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

## `del` and Garbage Collection
The `del` statement deletes variables, not objects. An object may be garbage collected as result of a del command, but only if the variable deleted holds **the last reference** to the object, or if the object becomes unreachable.

Each object keeps count of how many references point to it. As soon as that refcount reaches zero, the object is immediately destroyed: CPython calls the __del__ method on the object (if defined) and then frees the memory allocated to the object. 

We use `weakref.finalize` to register a callback function to be called when an object is destroyed.

In [15]:
import weakref

s1 = {1, 2, 3}
s2 = s1 # alias for {1, 2, 3}
def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1 # do not delete objects, only delete variables
print(ender.alive)
s2 = 'spam' # inplicitly call `bye`
print(ender.alive)

True
True
Gone with the wind...
False


## Weak References
Weak references to an object do not increase its reference count. Therefore, we say that a weak reference does not prevent the referent from being garbage collected.

If the object is alive, calling the weak reference returns it, otherwise `None` is returned.

In [16]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
print(wref())
a_set = {2, 3, 4} # no variable points to {0, 1}
print(wref())

{0, 1}
None


## The `weakref.WeakValueDictionary` Skit
The class `WeakValueDictionary` implements a mutable mapping where the **values** are weak references to objects. When a referred object is garbage collected elsewhere in the program, the corresponding key is automatically removed from `WeakValueDictionary`. 

In [17]:
import weakref

class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return f'Cheese({self.kind!r})'

stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese # The `stock` maps the name of the cheese to a weak reference to the cheese instance in the catalog.
print(sorted(stock.keys()))
del catalog
print(sorted(stock.keys())) # `cheese` has the last reference
del cheese
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']
[]


### Limitations of Weak references
Basic `list` and `dict` instances may not be referents of a weak reference, but a plain subclass of either can solve this problem easily

In [18]:
class MyList(list):
    """list subclass whose instances may be weakly referenced"""
a_list = list(range(10))
try:
    wref_to_list = weakref.ref(a_list)
except TypeError as e:
    print(e)

a_subclass = MyList(range(10))
wref_to_list = weakref.ref(a_subclass)
wref_to_list()

cannot create weak reference to 'list' object


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Some Tricks
for a tuple `t`, `t[:]` does not make a copy, but returns a reference to the same object. You also get a reference to the same tuple if you write `tuple(t)`

In [19]:
a = (1, 2)
a[:] is a

True